<h1>NBA 2K20 Database</h1>
<em>Aaron Wollman, Kelsey Richardson Blackwell, Will Huang, Albin Joseph</em>
<hr>

TODO: Insert project description here.

## Prerequisites

TODO: Writeup prerequisites for this code to run here.  Include any schema queries that need to be run, or any config.py files that need to be created.

## Setup

In order for the code to run, the dependencies in the next cell are required.

<em>Note that a config.py file is <b>required</b> for the next cell to run. 
    Follow the directions in Prerequisites section to create this file.</em>

In [ ]:
import pandas as pd
# TODO Other dependencies
from config import username, password

In [ ]:
csv_files = {
    "nba2k" = "data/nba2k20.csv"
    "player_stats" = "data/players_stats.csv"
}

## Extract

After the dependencies are setup, now the code will import the data to be worked on. Pandas will be used to import this data into DataFrames which will be cleaned up in the next section. Both files that will be imported are CSV files, which makes this step fairly easy.

### NBA 2K20 Statistics

This data contains player statistics from the videogame NBA 2K20. This videogame only contains data from the 2019 - 2020 NBA season.

In [ ]:
# TODO: Use pandas to import data/nba2k20.csv

### NBA Player Statistics

This data contains real-life player statistics for many seasons and leagues.

In [ ]:
# TODO: Use pandas to import data/players_stats.csv

## Transform

Now that the data has been loaded, it now needs to be cleaned up before it is loaded up into the database.

### NBA 2K20 Statistics

For the NBA 2K20 Statistics, it needs to be...

In [ ]:
# TODO: Clean up data, rename columns to match database, etc.

### NBA Player Statistics

For the NBA Player Statistics, the data needs to be...

In [ ]:
# TODO: Clean up data, rename columns to match database, etc.

### Merge & Reorganize Statistics

Now that the datasets are cleaned up, the tables need to b

## Load

Finally, the data can be loaded into the production database for any clients to potentially use. The production database is an SQL relational database with the following tables:
<ul>
    <li><em>Table</em> - Description</li>
</ul>
The database is structured in this way because...

### NBA 2K20 Statistics

In [ ]:
# TODO: Load up data into the production database.

### NBA Player Statistics

In [ ]:
# TODO: Load up data into the production database.

## Production

To test to make sure that this ETL project works correctly, run database/queries.sql.  The queries in this file will verify that the data was cleaned up correctly such that merges between tables work.